In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import yaml

import numpy as np
import pandas as pd

import tempfile
from arsenic import wrangle
from arsenic import plotting, stats

from PLBenchmarks import targets, ligands, edges

from tqdm.notebook import tqdm
import pint
unit_registry = pint.UnitRegistry()

import benchmarkpl
path = benchmarkpl.__path__[0]
targets.set_data_dir(path)

In [2]:
all_edges = pd.read_csv('03a_all_edges_all_ffs.csv', index_col=0)
all_edges.head()

,target,edge,ligandA,ligandB,unit,DDG_Exp.,dDDG_Exp.,DDG_OpenFF-1.0,dDDG_OpenFF-1.0,DDG_OpenFF-1.0_converged,...,error_GAFF2,abserror_GAFF2,error_cGenFF,abserror_cGenFF,error_Consensus_OpenFF_GAFF2_cGenFF,abserror_Consensus_OpenFF_GAFF2_cGenFF,error_Consensus_OpenFF_GAFF2,abserror_Consensus_OpenFF_GAFF2,error_Consensus_all,abserror_Consensus_all
jnk1_edge_17124-1_18631-1,jnk1,edge_17124-1_18631-1,17124-1,18631-1,kilocalories / mole,0.26,0.37,1.190000,0.096086,1.190000,...,1.071262,1.071262,0.516769,0.516769,0.646112,0.646112,0.784876,0.784876,1.222263,1.222263
jnk1_edge_17124-1_18634-1,jnk1,edge_17124-1_18634-1,17124-1,18634-1,kilocalories / mole,-0.33,0.29,0.580000,0.128639,0.580000,...,0.829522,0.829522,0.580956,0.580956,0.852556,0.852556,0.928604,0.928604,0.798413,0.798413
jnk1_edge_18626-1_18624-1,jnk1,edge_18626-1_18624-1,18626-1,18624-1,kilocalories / mole,0.38,0.21,0.556667,0.099301,0.556667,...,0.745717,0.745717,-0.265277,0.265277,0.309516,0.309516,0.616033,0.616033,0.446727,0.446727
jnk1_edge_18626-1_18625-1,jnk1,edge_18626-1_18625-1,18626-1,18625-1,kilocalories / mole,0.77,0.21,-0.030000,0.107462,-0.030000,...,-0.062543,0.062543,-0.294379,0.294379,-0.388337,0.388337,-0.155679,0.155679,0.143932,0.143932
jnk1_edge_18626-1_18627-1,jnk1,edge_18626-1_18627-1,18626-1,18627-1,kilocalories / mole,0.39,0.22,0.140000,0.046151,0.140000,...,0.042600,0.042600,-0.232256,0.232256,-0.020344,0.020344,0.064101,0.064101,-0.124060,0.124060


In [3]:
identifiers = [idx[4:] for idx in all_edges.columns if idx.startswith("DDG")]
identifiers

['Exp.',
 'OpenFF-1.0',
 'OpenFF-1.0_converged',
 'OpenFF-1.2',
 'OpenFF-1.2_converged',
 'OpenFF-2.0',
 'OpenFF-2.0_converged',
 'OPLS3e',
 'GAFF2',
 'cGenFF',
 'Consensus_OpenFF_GAFF2_cGenFF',
 'Consensus_OpenFF_GAFF2',
 'Consensus_all']

In [4]:
femap = {}
with tempfile.TemporaryDirectory() as tmpdirname:
    for target in tqdm(targets.target_dict):
        femap[target] = {}
        ligSet = ligands.LigandSet(target)
        for idx in tqdm(identifiers[1:], leave=False):
            with open(os.path.join(tmpdirname, f'{target}.csv'), 'w') as fo:
                fo.write('# Experimental block\n'\
                    '# Ligand, expt_DDG, expt_dDDG\n'
                    )
                df = ligSet.get_dataframe(columns=['name', 'DerivedMeasurement'])
                df.columns = df.columns.droplevel(0)
                df.index = df[''].apply(lambda x: x.replace('lig_', ''))
                df = df.drop(labels='', axis=1)
                df['dg'] = df['dg'].apply(lambda x: x.to('kilocalories / mole').magnitude)
                df['e_dg'] = df['e_dg'].apply(lambda x: x.to('kilocalories / mole').magnitude)
#                 print(df)
                fo.write(df.to_csv(header=None))
                fo.write('\n')

                fo.write('# Calculated block\n'\
                '# Ligand1,Ligand2, calc_DDG, calc_dDDG(MBAR), calc_dDDG(additional)\n'
                        )
                d = all_edges.loc[all_edges['target']==target,:]
                for key, row in d.iterrows():
                    if not np.isnan(row[f'DDG_{idx}']):
                        DDG = unit_registry.Quantity(row[f'DDG_{idx}'], row['unit']).to('kilocalories/mole').magnitude
                        dDDG = unit_registry.Quantity(row[f'dDDG_{idx}'], row['unit']).to('kilocalories/mole').magnitude
                        fo.write(f"{row['ligandA']}, {row['ligandB']}, {DDG}, {dDDG}, 0.0\n")
            filename=os.path.join(tmpdirname, f'{target}.csv')
            try:
                femap[target][idx] = wrangle.FEMap(filename)
            except Exception as e:
                print(f'Generating the FEMap for target {target} with data {idx} was not possible. {e}')

  0%|          | 0/22 [00:00<?, ?it/s]

/home/dhahn3/miniconda3/envs/off/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:1638: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


  0%|          | 0/12 [00:00<?, ?it/s]

Generating the FEMap for target jnk1 with data OpenFF-1.2 was not possible. division by zero
Generating the FEMap for target jnk1 with data OpenFF-1.2_converged was not possible. division by zero


  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

Graph is not connected enough to compute absolute values
Graph is not connected enough to compute absolute values
Graph is not connected enough to compute absolute values


  0%|          | 0/12 [00:00<?, ?it/s]

Graph is not connected enough to compute absolute values
Graph is not connected enough to compute absolute values
Graph is not connected enough to compute absolute values


  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
def get_dg_dataframe(femaps, idx=[], centralizing=False, shift=None):
    ref_idx='Exp.'
    df = pd.DataFrame(columns=['ligand', 'target', f'DG_{ref_idx}', f'dDG_{ref_idx}'] + [f'DG_{i}' for i in idx] + [f'dDG_{i}' for i in idx])
    for target_name, target_maps in femaps.items():
        ligSet = ligands.LigandSet(target_name).get_dataframe()
        ligSet.index=ligSet['name']
        for i in idx:
            if i in target_maps:
                graph = target_maps[i].graph
            else:
                continue
            if not target_maps[i].check_weakly_connected():
                print(f'{target_name} {i} weakly connected')
                continue
            n = [x[1]['name'] for x in graph.nodes(data=True)]
            x = np.asarray([x[1]['exp_DG'] for x in graph.nodes(data=True)])
            xerr = np.asarray([x[1]['exp_dDG'] for x in graph.nodes(data=True)])            
            y = np.asarray([x[1]['calc_DG'] for x in graph.nodes(data=True)])
            yerr = np.asarray([x[1]['calc_dDG'] for x in graph.nodes(data=True)])
            
            # centralising
            # this should be replaced by providing one experimental result
            if shift is None:
                shift = np.mean(x)
            if centralizing == True:
                x = x - np.mean(x) + shift
                y = y - np.mean(y) + shift 
            y = y + np.mean(x) - np.mean(y)
            for nn, xx, xxerr, yy, yyerr in zip(n, x, xerr, y, yerr):
                df.loc[f"{target_name}_lig_{nn}", "ligand"] = nn
                df.loc[f"{target_name}_lig_{nn}", f"DG_{i}"] = yy
                df.loc[f"{target_name}_lig_{nn}", f"dDG_{i}"] = yyerr
                df.loc[f"{target_name}_lig_{nn}", f"DG_{ref_idx}"] = xx
                df.loc[f"{target_name}_lig_{nn}", f"dDG_{ref_idx}"] = xxerr
                df.loc[f"{target_name}_lig_{nn}", f"target"] = target_name
    for i, col in df.iteritems():
        if i.startswith("DG") or i.startswith("dDG"):
            df[i] = df[i].astype('float')
    return df

In [6]:
all_ligands = get_dg_dataframe(femap, idx=identifiers[1:])
all_ligands.head()

pde2 OpenFF-2.0_converged weakly connected
eg5 OpenFF-2.0_converged weakly connected
pde10 OpenFF-1.0_converged weakly connected
pde10 OpenFF-2.0_converged weakly connected
shp2 OpenFF-1.0_converged weakly connected
shp2 OpenFF-1.2_converged weakly connected
shp2 OpenFF-2.0_converged weakly connected
syk OpenFF-1.0_converged weakly connected
syk OpenFF-1.2_converged weakly connected
syk OpenFF-2.0_converged weakly connected


,ligand,target,DG_Exp.,dDG_Exp.,DG_OpenFF-1.0,DG_OpenFF-1.0_converged,DG_OpenFF-1.2,DG_OpenFF-1.2_converged,DG_OpenFF-2.0,DG_OpenFF-2.0_converged,...,dDG_OpenFF-1.2,dDG_OpenFF-1.2_converged,dDG_OpenFF-2.0,dDG_OpenFF-2.0_converged,dDG_OPLS3e,dDG_GAFF2,dDG_cGenFF,dDG_Consensus_OpenFF_GAFF2_cGenFF,dDG_Consensus_OpenFF_GAFF2,dDG_Consensus_all
jnk1_lig_17124-1,17124-1,jnk1,-9.76,0.29,-10.498849,-10.410240,NaN,NaN,-11.647484,-11.641639,...,NaN,NaN,0.126579,0.127663,0.052150,0.193056,0.174543,0.245633,0.299317,0.103591
jnk1_lig_18631-1,18631-1,jnk1,-9.50,0.23,-9.208652,-9.114652,NaN,NaN,-9.480523,-9.474986,...,NaN,NaN,0.094876,0.096155,0.049214,0.402302,0.168230,0.325155,0.320194,0.127863
jnk1_lig_18634-1,18634-1,jnk1,-10.09,0.04,-10.098436,-10.020989,NaN,NaN,-11.048733,-11.048163,...,NaN,NaN,0.126496,0.127282,0.044415,0.132499,0.185769,0.223787,0.261344,0.093516
jnk1_lig_18626-1,18626-1,jnk1,-8.95,0.14,-8.185239,-8.133063,NaN,NaN,-8.077112,-8.077298,...,NaN,NaN,0.080243,0.081041,0.032254,0.107774,0.141012,0.165191,0.146609,0.066499
jnk1_lig_18624-1,18624-1,jnk1,-8.57,0.15,-7.995236,-7.936688,NaN,NaN,-7.067734,-7.068933,...,NaN,NaN,0.078940,0.079947,0.034820,0.116900,0.140377,0.178736,0.151406,0.068561


In [7]:
all_ligands.to_csv('03b_all_ligands_all_ffs.csv')